# Finetuning a LLM in Macbook Pro
### System Configuration
- Macbook Pro M4 Pro
- 48GB RAM
- GPU Cores: 20
- Metal Support: Metal 4

### MPS (Metal Performance Shaders): 
- Apple-provided GPU backend that PyTorch can use on Apple Silicon. Using MPS speeds training/inference compared to CPU.
- PyTorch tensors and model weights can be moved to device="mps"
- Forward + backward passes happen on the GPU → much faster than CPU


### Environment Setup

```
uv env .venv --python=3.12
source .venv/bin/activate
uv add ipykernel jupyter
uv add torch torchvision torchaudio
uv add transformers accelerate datasets huggingface_hub safetensors

# Register your venv as a fresh kernel
python -m ipykernel install --user \
  --name=gemma270m \
  --display-name "Gemma 270M (venv)"
```

Restart IDE & select kernel: Gemma 270M (venv)

In [197]:
# Check whether MPS is available to train on Apple GPU (Metal):
import torch
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

MPS available: True
MPS built: True


### Configure Hugging Face authentication
In terminal run
```
huggingface-cli login
```
paste your token from https://huggingface.co/settings/tokens)

### Move model to GPU (MPS)
- We want PyTorch to use MPS (Metal Performance Shaders) for GPU acceleration. Mac M4 Pro has 20 GPU cores.
- `model.to(device)` moves all weights to GPU memory
- Forward pass (computing predictions) + backward pass (gradients) happen on GPU
- Using CPU is much slower because PyTorch cannot parallelize large matrix multiplications as efficiently

In [198]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

### Load the model and the tokenizer
- Tokenizer:
  - Maps words/subwords → integers
  - Handles special tokens like <pad>, <eos>
  - Ensures input sequences are exactly what the model expects

- Model:
  - PyTorch object containing all 270M weights
  - Decoder-only transformer: takes token IDs → predicts next token logits
  - from_pretrained downloads weights, config, and merges all necessary files

In [199]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/functiongemma-270m-it"

# Load tokenizer
# Fix the tokenizer regex bug manually to avoid the TypeError crash
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.fix_mistral_regex = True

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, # Crucial for Gemma 270M
    device_map=device
)

### Prepare Dataset

In [200]:
import json
from datasets import Dataset

# --- Tool Definition for a Travel Planner ---
def search_web(query: str) -> str:
    """
    Search the web using DuckDuckGo.

    Args:
        query: query string
    Returns:
        Top web result as string
    """
    return f"Search result for: {query}"


In [201]:
simple_tool_calling = [
    {
        "user_content": "Best places to visit in Switzerland for 5 days",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best places to visit in switzerland for 5 days"}'
    },
    {
        "user_content": "Family-friendly 5-day Switzerland trip with kids",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "family-friendly 5-day switzerland trip with kids"}'
    },
    {
        "user_content": "Best time to visit Japan for cherry blossoms",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best time to visit japan for cherry blossoms"}'
    },
    {
        "user_content": "Cheap transport passes in Switzerland for tourists",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "cheap transport passes in switzerland for tourists"}'
    },
    {
        "user_content": "Best scenic train routes in Switzerland",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best scenic train routes in switzerland"}'
    },
    {
        "user_content": "Top-rated car rental in Los Angeles",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top-rated car rental in los angeles"}'
    },
    {
        "user_content": "Relaxed 7-day Italy itinerary covering Rome and Florence",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "relaxed 7-day italy itinerary covering rome and florence"}'
    },
    {
        "user_content": "Where to go in Europe in October for fall colors",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "where to go in europe in october for fall colors"}'
    },
    {
        "user_content": "Switzerland vs Austria for a scenic trip",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "switzerland vs austria for a scenic trip"}'
    },
    {
        "user_content": "Things to do around Zurich",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "things to do around zurich"}'
    },
    {
        "user_content": "Hiking trails near Interlaken for beginners",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "hiking trails near interlaken for beginners"}'
    },
    {
        "user_content": "Best sushi spots in Tokyo near Shibuya",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best sushi spots in tokyo near shibuya"}'
    },
    {
        "user_content": "How to use the Swiss Travel Pass",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to use the swiss travel pass"}'
    },
    {
        "user_content": "Is Eurail pass worth it for Italy?",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "is eurail pass worth it for italy?"}'
    },
    {
        "user_content": "Budget-friendly hotels in Manhattan for a week",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "budget-friendly hotels in manhattan for a week"}'
    },
    {
        "user_content": "Top 10 things to do in Amsterdam",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top 10 things to do in amsterdam"}'
    },
    {
        "user_content": "Safety tips for solo female travelers in Iceland",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "safety tips for solo female travelers in iceland"}'
    },
    {
        "user_content": "Best ski resorts in the Swiss Alps for intermediates",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best ski resorts in the swiss alps for intermediates"}'
    },
    {
        "user_content": "Vegetarian food guide for Bangkok",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "vegetarian food guide for bangkok"}'
    },
    {
        "user_content": "How to book Shinkansen tickets in advance",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to book shinkansen tickets in advance"}'
    },
    {
        "user_content": "Hidden gems in the Amalfi Coast",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "hidden gems in the amalfi coast"}'
    },
    {
        "user_content": "Packing list for a 10-day trip to Scandinavia in winter",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "packing list for a 10-day trip to scandinavia in winter"}'
    },
    {
        "user_content": "Best museums to visit in Paris",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best museums to visit in paris"}'
    },
    {
        "user_content": "Day trip from Munich to Neuschwanstein Castle",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "day trip from munich to neuschwanstein castle"}'
    },
    {
        "user_content": "Affordable overwater bungalows in the Maldives",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "affordable overwater bungalows in the maldives"}'
    },
    {
        "user_content": "Wine tasting tours in Tuscany",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "wine tasting tours in tuscany"}'
    },
    {
        "user_content": "How to get from Heathrow to Central London",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to get from heathrow to central london"}'
    },
    {
        "user_content": "Top beaches in Crete, Greece",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top beaches in crete, greece"}'
    },
    {
        "user_content": "Cultural etiquette tips for visiting Morocco",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "cultural etiquette tips for visiting morocco"}'
    },
    {
        "user_content": "Best time to visit New Zealand for hiking",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best time to visit new zealand for hiking"}'
    },
    {
        "user_content": "Family activities in Singapore for a 3-day stopover",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "family activities in singapore for a 3-day stopover"}'
    },
    {
        "user_content": "Nightlife guide for Berlin",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "nightlife guide for berlin"}'
    },
    {
        "user_content": "Best street food markets in Mexico City",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best street food markets in mexico city"}'
    },
    {
        "user_content": "How to see the Northern Lights in Norway",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to see the northern lights in norway"}'
    },
    {
        "user_content": "Romantic getaways in the South of France",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "romantic getaways in the south of france"}'
    },
    {
        "user_content": "Exploring the Scottish Highlands by car",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the scottish highlands by car"}'
    },
    {
        "user_content": "Best surfing spots in Portugal",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best surfing spots in portugal"}'
    },
    {
        "user_content": "What to see in the Grand Canyon in one day",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "what to see in the grand canyon in one day"}'
    },
    {
        "user_content": "Visiting the Pyramids of Giza: A complete guide",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "visiting the pyramids of giza: a complete guide"}'
    },
    {
        "user_content": "Travel insurance for senior citizens",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "travel insurance for senior citizens"}'
    },
    {
        "user_content": "Best historical sites in Kyoto",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best historical sites in kyoto"}'
    },
    {
        "user_content": "Top waterfalls to visit in Bali",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top waterfalls to visit in bali"}'
    },
    {
        "user_content": "How to plan a road trip across the Garden Route, South Africa",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to plan a road trip across the garden route, south africa"}'
    },
    {
        "user_content": "Best coffee shops in Vienna",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best coffee shops in vienna"}'
    },
    {
        "user_content": "Whale watching tours in Vancouver",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "whale watching tours in vancouver"}'
    },
    {
        "user_content": "Visiting the Louvre: How to avoid the crowds",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "visiting the louvre: how to avoid the crowds"}'
    },
    {
        "user_content": "Best places for stargazing in Chile",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best places for stargazing in chile"}'
    },
    {
        "user_content": "Travel guide for the Galápagos Islands",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "travel guide for the galápagos islands"}'
    },
    {
        "user_content": "Exploring the ancient city of Petra",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the ancient city of petra"}'
    },
    {
        "user_content": "Best time for a safari in Kenya",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best time for a safari in kenya"}'
    },
    {
        "user_content": "Cheap hostels in Barcelona with good vibes",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "cheap hostels in barcelona with good vibes"}'
    },
    {
        "user_content": "How to navigate the Tokyo subway",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to navigate the tokyo subway"}'
    },
    {
        "user_content": "Best rooftop bars in Bangkok",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best rooftop bars in bangkok"}'
    },
    {
        "user_content": "Visiting the Sagrada Familia: Ticket tips",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "visiting the sagrada familia: ticket tips"}'
    },
    {
        "user_content": "Exploring the fjords of New Zealand",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the fjords of new zealand"}'
    },
    {
        "user_content": "Best Christmas markets in Germany",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best christmas markets in germany"}'
    },
    {
        "user_content": "How to get a visa for Vietnam",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to get a visa for vietnam"}'
    },
    {
        "user_content": "Top surfing beaches in Bali",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top surfing beaches in bali"}'
    },
    {
        "user_content": "Backpacking through Southeast Asia: A budget guide",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "backpacking through southeast asia: a budget guide"}'
    },
    {
        "user_content": "Best luxury hotels in Dubai",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best luxury hotels in dubai"}'
    },
    {
        "user_content": "Exploring the ruins of Machu Picchu",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the ruins of machu picchu"}'
    },
    {
        "user_content": "Best time to visit the Great Barrier Reef",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best time to visit the great barrier reef"}'
    },
    {
        "user_content": "How to travel from Lisbon to Porto by train",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to travel from lisbon to porto by train"}'
    },
    {
        "user_content": "Top things to do in Lisbon for 3 days",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top things to do in lisbon for 3 days"}'
    },
    {
        "user_content": "Best viewpoints in Santorini",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best viewpoints in santorini"}'
    },
    {
        "user_content": "Visiting the Blue Lagoon in Iceland: Is it worth it?",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "visiting the blue lagoon in iceland: is it worth it?"}'
    },
    {
        "user_content": "Street food guide for Seoul",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "street food guide for seoul"}'
    },
    {
        "user_content": "Best hiking trails in Yosemite National Park",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best hiking trails in yosemite national park"}'
    },
    {
        "user_content": "How to plan a weekend in Prague",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to plan a weekend in prague"}'
    },
    {
        "user_content": "Exploring the canals of Venice without the crowds",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the canals of venice without the crowds"}'
    },
    {
        "user_content": "Best places to stay in Lake Como",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best places to stay in lake como"}'
    },
    {
        "user_content": "Family-friendly resorts in the Algarve",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "family-friendly resorts in the algarve"}'
    },
    {
        "user_content": "How to travel on a budget in Norway",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to travel on a budget in norway"}'
    },
    {
        "user_content": "Best photo spots in Cappadocia",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best photo spots in cappadocia"}'
    },
    {
        "user_content": "Visiting the Taj Mahal: Sunrise or sunset?",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "visiting the taj mahal: sunrise or sunset?"}'
    },
    {
        "user_content": "Exploring the islands of Croatia by boat",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the islands of croatia by boat"}'
    },
    {
        "user_content": "Best brunch spots in Melbourne",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best brunch spots in melbourne"}'
    },
    {
        "user_content": "How to visit the Vatican Museums",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to visit the vatican museums"}'
    },
    {
        "user_content": "Exploring the wine regions of Australia",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the wine regions of australia"}'
    },
    {
        "user_content": "Best time to visit New York City",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best time to visit new york city"}'
    },
    {
        "user_content": "Top things to do in Montreal in winter",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top things to do in montreal in winter"}'
    },
    {
        "user_content": "How to get around in Istanbul",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to get around in istanbul"}'
    },
    {
        "user_content": "Best beaches in the Seychelles",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best beaches in the seychelles"}'
    },
    {
        "user_content": "Visiting the Angkor Wat temple complex",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "visiting the angkor wat temple complex"}'
    },
    {
        "user_content": "Exploring the colorful streets of Cartagena",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the colorful streets of cartagena"}'
    },
    {
        "user_content": "Best surfing in Costa Rica",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best surfing in costa rica"}'
    },
    {
        "user_content": "How to plan a trip to the Amazon Rainforest",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to plan a trip to the amazon rainforest"}'
    },
    {
        "user_content": "Top landmarks in Washington D.C.",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top landmarks in washington d.c."}'
    },
    {
        "user_content": "Best seafood in San Francisco",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best seafood in san francisco"}'
    },
    {
        "user_content": "Exploring the castles of the Loire Valley",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the castles of the loire valley"}'
    },
    {
        "user_content": "How to visit the Great Wall of China",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to visit the great wall of china"}'
    },
    {
        "user_content": "Best time for a desert safari in Jordan",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best time for a desert safari in jordan"}'
    },
    {
        "user_content": "Exploring the underground city of Derinkuyu",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the underground city of derinkuyu"}'
    },
    {
        "user_content": "Best places for a spa retreat in Bali",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best places for a spa retreat in bali"}'
    },
    {
        "user_content": "How to travel from Vienna to Budapest",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to travel from vienna to budapest"}'
    },
    {
        "user_content": "Top things to do in Budapest for a weekend",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "top things to do in budapest for a weekend"}'
    },
    {
        "user_content": "Best street art tours in Berlin",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best street art tours in berlin"}'
    },
    {
        "user_content": "Exploring the national parks of Utah",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "exploring the national parks of utah"}'
    },
    {
        "user_content": "How to see the lavender fields in Provence",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "how to see the lavender fields in provence"}'
    },
    {
        "user_content": "Best winter sun destinations in Europe",
        "tool_name": "search_web",
        "tool_arguments": '{"query": "best winter sun destinations in europe"}'
    }
]
print(f"Total samples: {len(simple_tool_calling)}")


Total samples: 100


In [202]:
import json
from datasets import Dataset
from transformers.utils import get_json_schema

# 1. Prepare Tools - we use separators to remove whitespace and save tokens
TOOLS = [get_json_schema(search_web)]
TOOL_JSON_STRING = json.dumps(TOOLS[0], separators=(',', ':'))

# 2. Strict Formatting Function
# This replaces your "create_conversation" function
def format_training_example(sample):
    """
    Constructs the exact mechanical string sequence for FunctionGemma 270M.
    This ensures the model learns the exact sequence of tokens.
    """
    # The 'developer' prompt must use this EXACT wording as a trigger
    developer_msg = "You are a model that can do function calling with the following functions"
    
    # Extract arguments - ensure they are formatted as a dict for the escape logic
    # Handles both string and dict inputs for robustness
    args_dict = json.loads(sample["tool_arguments"]) if isinstance(sample["tool_arguments"], str) else sample["tool_arguments"]
    query_val = args_dict.get("query", "")

    # The model response must start with the 'call:' prefix
    target_call = f"<start_function_call>call:{sample['tool_name']}{{query:<escape>{query_val}<escape>}}<end_function_call>"
    
    # Combine into a single raw text string
    full_text = (
        f"<bos><start_of_turn>developer\n{developer_msg}"
        f"<start_function_declaration>{TOOL_JSON_STRING}<end_function_declaration><end_of_turn>\n"
        f"<start_of_turn>user\n{sample['user_content']}<end_of_turn>\n"
        f"<start_of_turn>model\n{target_call}<end_of_turn>"
    )
    
    return {"text": full_text}

# 3. Create and Map Dataset
# Convert your list 'simple_tool_calling' to a Hugging Face dataset
dataset = Dataset.from_list(simple_tool_calling)

# Use our new strict formatter and remove the old columns to keep only 'text'
dataset = dataset.map(format_training_example)

# 4. Split for validation
dataset = dataset.train_test_split(test_size=0.5, shuffle=True)

# --- VERIFICATION ---
print("--- Prepared Training String Example ---")
print(dataset["train"][0]["text"])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

--- Prepared Training String Example ---
<bos><start_of_turn>developer
You are a model that can do function calling with the following functions<start_function_declaration>{"type":"function","function":{"name":"search_web","description":"Search the web using DuckDuckGo.","parameters":{"type":"object","properties":{"query":{"type":"string","description":"query string"}},"required":["query"]},"return":{"type":"string","description":"Top web result as string"}}}<end_function_declaration><end_of_turn>
<start_of_turn>user
Visiting the Angkor Wat temple complex<end_of_turn>
<start_of_turn>model
<start_function_call>call:search_web{query:<escape>visiting the angkor wat temple complex<escape>}<end_function_call><end_of_turn>


### Test Before Finetuning

In [203]:
def check_success_rate():
    success_count = 0
    # We loop through the test split which now contains 'user_content' and 'tool_name'
    for idx, item in enumerate(dataset['test']):
        
        # 1. Reconstruct the inference messages
        # We use the 'developer' role to match the training format
        messages = [
            {"role": "developer", "content": "You are a model that can do function calling with the following functions"},
            {"role": "user", "content": item["user_content"]},
        ]

        # 2. Prepare inputs using chat template
        inputs = tokenizer.apply_chat_template(
            messages,
            tools=TOOLS,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt"
        )

        # 3. Generate model output
        # Use do_sample=False for consistent evaluation results
        out = model.generate(
            **inputs.to(model.device),
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=128,
            do_sample=False 
        )

        # 4. Decode output
        output = tokenizer.decode(
            out[0][len(inputs["input_ids"][0]):],
            skip_special_tokens=False
        )

        print(f"{idx+1}. Prompt: {item['user_content']}")
        print(f"   Output: {output}")

        # 5. Check success against the original 'tool_name'
        expected_tool = item['tool_name']

        if expected_tool in output:
            print("   -> ✅ correct!")
            success_count += 1
        else:
            print(f"   -> ❌ wrong (expected '{expected_tool}' missing)")

    print(f"\nOverall Success: {success_count} / {len(dataset['test'])}")

check_success_rate()

1. Prompt: Best luxury hotels in Dubai
   Output: I cannot assist with recommending exclusive or high-quality luxury hotels in Dubai. My current capabilities are focused on searching the web for specific queries using DuckGo. I cannot browse or recommend specific hotels.<end_of_turn>
   -> ❌ wrong (expected 'search_web' missing)
2. Prompt: Packing list for a 10-day trip to Scandinavia in winter
   Output: I am sorry, but I cannot assist with planning travel logistics or packing lists for a trip to Scandinavia. My current capabilities are limited to assisting with searching the web using specific tools. I cannot generate or manage travel planning or packing lists.<end_of_turn>
   -> ❌ wrong (expected 'search_web' missing)
3. Prompt: How to get a visa for Vietnam
   Output: I cannot assist with visa requirements or advice for obtaining a visa for Vietnam. My current capabilities are limited to assisting with web searching and searching the web.<end_of_turn>
   -> ❌ wrong (expected 'searc

### Parameter-Efficient Fine-Tuning (PEFT) using Low-Rank Adaptation (LoRA)
- PEFT (Parameter-Efficient Fine-Tuning) is a method to adapt large pretrained models (like LLMs) to new tasks without retraining all model parameters.
- Instead of updating billions of weights, most of the base model is frozen, which greatly reduces training time, memory usage, and hardware requirements.
- LoRA (Low-Rank Adaptation) is a popular PEFT technique that adds small, trainable low-rank matrices to selected layers of the model.
- During training, only these added LoRA parameters are updated, while the original model weights remain unchanged.
- This allows the model to learn task-specific behavior efficiently, often achieving performance close to full fine-tuning.



In [204]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# 1. Setup LoRA - Updated for Gemma architecture
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# 2. Strict Tokenization Function
def prepare_example(sample):
    """
    Manually constructs the tokens to ensure labels are masked correctly.
    Loss is ONLY computed on the assistant's function call.
    """
    developer_msg = "You are a model that can do function calling with the following functions"
    
    # Extract data from the flat structure
    args_dict = json.loads(sample["tool_arguments"]) if isinstance(sample["tool_arguments"], str) else sample["tool_arguments"]
    query_val = args_dict.get("query", "")
    
    # Construct the Prompt (System + User)
    prompt_text = (
        f"<bos><start_of_turn>developer\n{developer_msg}"
        f"<start_function_declaration>{json.dumps(TOOLS[0], separators=(',', ':'))}<end_function_declaration><end_of_turn>\n"
        f"<start_of_turn>user\n{sample['user_content']}<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )
    
    # Construct the Target (Assistant Call)
    target_text = f"<start_function_call>call:{sample['tool_name']}{{query:<escape>{query_val}<escape>}}<end_function_call><end_of_turn>"

    # Tokenize
    prompt_ids = tokenizer.encode(prompt_text, add_special_tokens=False)
    target_ids = tokenizer.encode(target_text, add_special_tokens=False)

    # Combine
    input_ids = prompt_ids + target_ids
    # Mask the prompt: we only want the model to learn to predict the target_ids
    labels = ([-100] * len(prompt_ids)) + target_ids

    return {"input_ids": input_ids, "labels": labels}

# 3. Process the dataset splits
# We use .map to apply the tokenization but keep the structure simple
train_ds = dataset["train"].map(prepare_example, remove_columns=dataset["train"].column_names)
eval_ds  = dataset["test"].map(prepare_example, remove_columns=dataset["test"].column_names)

# 4. Data Collator (Standard for Causal LM)
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True)

# 5. Training Arguments
training_args = TrainingArguments(
    output_dir="./functiongemma_results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    bf16=True, # Ensure your Mac supports bf16, otherwise use fp16=True
    logging_steps=1,
    num_train_epochs=10, # Increased for the tiny model to stabilize
    weight_decay=0.01,
    save_strategy="no",
    report_to="none"
)

# 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
)

# 7. Execute Training
trainer.train()

# 8. Save
model.save_pretrained("./functiongemma_lora_adapter")
print("LoRA adapter saved successfully.")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
1,4.792200
2,4.478500
3,4.098900
4,3.135800
5,3.129400
6,2.638600
7,2.180700
8,2.234600
9,1.459400
10,1.058600


LoRA adapter saved successfully.


### Merge LoRA Adapter and Base Model

In [205]:
#Load base model fresh
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_name = "google/functiongemma-270m-it"
lora_path = "./functiongemma_lora_adapter"
merged_output_path = "./functiongemma_merged"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float32,
    device_map=None
)

#Load LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    lora_path
)

#Merge and unload LoRA
merged_model = model.merge_and_unload()

#Save merged model to get standalone fine-tuned model
merged_model.save_pretrained(merged_output_path)
tokenizer.save_pretrained(merged_output_path)


('./functiongemma_merged/tokenizer_config.json',
 './functiongemma_merged/special_tokens_map.json',
 './functiongemma_merged/chat_template.jinja',
 './functiongemma_merged/tokenizer.json')

### Test After Finetuning by loading the merged model (no PEFT anymore)

In [206]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./functiongemma_merged")

model = AutoModelForCausalLM.from_pretrained(
    "./functiongemma_merged",
    dtype=torch.float32
)

model.to(device)
model.eval()

check_success_rate()

The tokenizer you are loading from './functiongemma_merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


1. Prompt: Best luxury hotels in Dubai
   Output: <start_function_call>call:search_web{query:<escape>best luxury hotels in dubai<escape>}<end_function_call><end_of_turn>
   -> ✅ correct!
2. Prompt: Packing list for a 10-day trip to Scandinavia in winter
   Output: <start_function_call>call:search_web{query:<escape>packing list for a 10-day trip to Scandinavia in winter<escape>}<end_function_call><end_of_turn>
   -> ✅ correct!
3. Prompt: How to get a visa for Vietnam
   Output: <start_function_call>call:search_web{query:<escape>how to get a visa for vietnam<escape>}<end_function_call><end_of_turn>
   -> ✅ correct!
4. Prompt: Vegetarian food guide for Bangkok
   Output: <start_function_call>call:search_web{query:<escape>vegetarian food guide for Bangkok<escape>}<end_function_call><end_of_turn>
   -> ✅ correct!
5. Prompt: How to see the Northern Lights in Norway
   Output: <start_function_call>call:search_web{query:<escape>how to see the northern lights in norway<escape>}<end_function_cal